In [11]:
import pandas as pd
import argparse as args
import numpy as np

#%load_ext rpy2.ipython


# Alzheimer's Disease formatting:


#### Reading data:

In [2]:
alzheimer = pd.DataFrame(pd.read_csv("~/alzheimersproject/1_raw_data/alzheimer_wightman_freq", 
                delim_whitespace=True, header=0, na_values='NA'))

alzheimer

,CHR,POS_GRCh37,Effect_allele,Non_Effect_allele,Z,P,N,Effect_allele_freq
0,1,13668,A,G,-0.337420,0.7358,74004,0.005810
1,1,14506,A,G,-0.511216,0.6092,74004,0.005414
2,1,14773,T,C,1.371240,0.1703,74004,0.017350
3,1,14860,T,G,-1.275874,0.2020,74004,0.011770
4,1,17101,C,G,-0.471057,0.6376,74004,0.017450
...,...,...,...,...,...,...,...,...
12688334,22,51239578,C,T,-0.768483,0.4422,74004,0.005218
12688335,22,51239584,C,G,-0.768483,0.4422,74004,0.005218
12688336,22,51239586,G,T,-0.737201,0.4610,74004,0.006382
12688337,22,51242557,T,A,-1.158893,0.2465,74004,0.061110


From the table above, it is clear, that the dataset is only missing the `SNP`column.   
To add this, we need a reference file (dbSNP v150 for hg19 was used), which is downoaded using the following:  
`wget https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b150_GRCh37p13/VCF/All_20170710.vcf.gz`

The reference file was then unzipped and renamed hg19_ref. 

#### Merging reference file with alzheimer in order to extract the rsIDs:

In [1]:
%%bash

# 1. Creating chr_pos column in reference and alzheimer file:
awk -v OFS='\t' '{print $1"_"$2, $0}' ~/alzheimersproject/1_raw_data/hg19_ref > hg19_ref_CHRPOS
awk -v OFS='\t' '{print $1"_"$2, $0}' ~/alzheimersproject/1_raw_data/alzheimer_wightman_freq > alzheimer_wightman_CHRPOS

# 2. Creating files that only contain the chr_pos column:
cut -f1 hg19_ref_CHRPOS > hg19_ref_CHRPOS_col1
cut -f1 alzheimer_wightman_CHRPOS > alzheimer_wightman_CHRPOS_col1

# 3. Creating files that only contain unique chr_pos variants names:
sort alzheimer_wightman_CHRPOS_col1 | uniq -u > alzheimer_wightman_CHRPOS_col1_uniq
sort hg19_ref_CHRPOS_col1 | uniq -u > hg19_ref_CHRPOS_col1_uniq

# 4. Creating new files with the duplicate chr_pos variant names removed:
awk 'NR==FNR {FILE1[$1]=$0; next} ($1 in FILE1) {print $0}' alzheimer_wightman_CHRPOS_col1_uniq alzheimer_wightman_CHRPOS > alzheimer_wightman_CHRPOS_uniq
awk 'NR==FNR {FILE1[$1]=$0; next} ($1 in FILE1) {print $0}' hg19_ref_CHRPOS_col1_uniq hg19_ref_CHRPOS > hg19_ref_CHRPOS_uniq

# 5. Adding rsIDs from reference to alzheimer file:
awk -v OFS='\t' 'NR==FNR { FILE1[$1]=$4; next} ($1 in FILE1) {print FILE1[$1], $0}' hg19_ref_CHRPOS_uniq alzheimer_wightman_CHRPOS_uniq > alzheimer_wightman_rsID

#### Creating file with removed SNPs due to missing rsID:

In [2]:
%%bash

# Removing the rsID column to make the two files identical in regard to columns:
awk -v OFS='\t' '{ print $2, $3, $4, $5, $6, $7, $8, $9, $10 }' alzheimer_wightman_rsID > alzheimer_hg19

# Tab seperate file:
cat alzheimer_wightman_CHRPOS_uniq | tr ' ' '\t' > alzheimer_wightman_CHRPOS_uniq_tab

# Comparing the two file and only print the rows that are unique for alzheimer_wightman_CHRPOS_uniq:
comm -2 -3 <(sort alzheimer_wightman_CHRPOS_uniq_tab) <(sort alzheimer_hg19) > alzheimer_removed_snps
wc -l alzheimer_removed_snps

# Making a file that only contain the SNPs that were significant (P<0.05):
#awk '$7 < 0.05' alzheimer_removed_snps > alzheimer_sig_removed_snps
#wc -l alzheimer_sig_removed_snps

287373 alzheimer_removed_snps


#### Trying to find rsIDs for the significant SNPs that were removed:

In [3]:
%%bash

# 1. Creating chr_pos column in reference file (here Jansen data):
awk -v OFS='\t' '{print $2"_"$3, $0}' ~/alzheimersproject/1_raw_data/alzheimer_jansen > alzheimer_jansen_CHRPOS

# 2. Creating files that only contain the chr_pos column:
cut -f1 alzheimer_jansen_CHRPOS > alzheimer_jansen_CHRPOS_col1
cut -f1 alzheimer_removed_snps > alzheimer_removed_snps_col1

# 3. Creating files that only contain unique chr_pos variants names:
sort alzheimer_removed_snps_col1 | uniq -u > alzheimer_removed_snps_col1_uniq
sort alzheimer_jansen_CHRPOS_col1 | uniq -u > alzheimer_jansen_CHRPOS_col1_uniq

# 4. Creating new files with the duplicate chr_pos variant names removed:
awk 'NR==FNR {FILE1[$1]=$0; next} ($1 in FILE1) {print $0}' alzheimer_removed_snps_col1_uniq alzheimer_removed_snps > alzheimer_removed_snps_uniq
awk 'NR==FNR {FILE1[$1]=$0; next} ($1 in FILE1) {print $0}' alzheimer_jansen_CHRPOS_col1_uniq alzheimer_jansen_CHRPOS > alzheimer_jansen_CHRPOS_uniq

# 5. Adding rsIDs from reference to removed SNPs file:
awk -v OFS='\t' 'NR==FNR { FILE1[$1]=$7; next} ($1 in FILE1) {print FILE1[$1], $0}' alzheimer_jansen_CHRPOS_uniq alzheimer_removed_snps_uniq > alzheimer_removed_snps_rsID
wc -l alzheimer_removed_snps_rsID

117277 alzheimer_removed_snps_rsID


#### Adding the SNPs that were removed, but that are now containing an rsID:

In [4]:
%%bash 

cat alzheimer_wightman_rsID alzheimer_removed_snps_rsID > alzheimer_hg19_jansen_rsid

#### Adding beta and se:

In [40]:
%%bash

awk -v OFS='\t' 'BEGIN { print "SNP", "CHR", "BP", "A1", "A2", "Z", "P", "N", "FREQ", "BETA", "SE"} { print $1, $3, $4, $5, $6, $7, $8, $9, $10, $11 = ($7/sqrt($9/(2*$10(1-$10)))), $12 = (sqrt(1/($9/(2*$10(1-$10))))) }' alzheimer_hg19_jansen_rsid > alzheimer_all_cols

#### Removing potential non-rsIDs in the SNP column:

In [41]:
%%bash

awk -v OFS='\t' 'NR == 1 || $1 ~ "rs"' alzheimer_all_cols > alzheimer_all_cols_onlyrs

#### Removing possible duplicates i the file:

In [42]:
%%bash

awk '!seen[$1]++' alzheimer_all_cols_onlyrs > alzheimer_onlyrs_uniq

#### Checking if it worked:

In [43]:
alzheimer_formatted = pd.DataFrame(pd.read_csv("~/alzheimersproject/2_formatting/alzheimer_onlyrs_uniq", 
                delim_whitespace=True, na_values='NA'))

alzheimer_formatted

,SNP,CHR,BP,A1,A2,Z,P,N,FREQ,BETA,SE
0,rs2691328,1,13668,A,G,-0.337420,0.735800,74004,0.005810,-0.000134,0.000396
1,rs878915777,1,14773,T,C,1.371240,0.170300,74004,0.017350,0.000939,0.000685
2,rs533499096,1,14860,T,G,-1.275874,0.202000,74004,0.011770,-0.000720,0.000564
3,rs867691030,1,17147,A,G,0.085329,0.932000,74004,0.016020,0.000056,0.000658
4,rs373918278,1,17765,A,G,0.093886,0.925200,74004,0.006307,0.000039,0.000413
...,...,...,...,...,...,...,...,...,...,...,...
12451130,rs10817096,9,99865774,C,A,1.007255,0.313812,8313,0.141105,0.005869,0.005826
12451131,rs10817129,9,99880920,C,G,0.806652,0.419867,262579,0.166765,0.000909,0.001127
12451132,rs139328556,9,99906691,G,A,-0.533539,0.593661,760064,0.025099,-0.000137,0.000257
12451133,rs371129421,9,99934073,A,T,0.491369,0.623166,8313,0.025944,0.001228,0.002498


#### Checking for NaN, NA, and inf.:

In [45]:
# 1. Checking for NaN:
print(alzheimer_formatted.isnull().sum())

print('\n------------------------------\n')

# 2. Checking for NA:
print(alzheimer_formatted.isna().sum())

print('\n------------------------------\n') 

# 3. Checking for inf:
print((alzheimer_formatted.isin([np.inf, -np.inf])).sum())


SNP     0
CHR     0
BP      0
A1      0
A2      0
Z       0
P       0
N       0
FREQ    0
BETA    0
SE      0
dtype: int64

------------------------------

SNP     0
CHR     0
BP      0
A1      0
A2      0
Z       0
P       0
N       0
FREQ    0
BETA    0
SE      0
dtype: int64

------------------------------

SNP      0
CHR      0
BP       0
A1       0
A2       0
Z       31
P        0
N        0
FREQ     0
BETA     2
SE       0
dtype: int64


#### Checking for inf/-inf values and P=0:

In [16]:
%%bash

awk -v OFS='\t' 'NR == 1 || $7 == "0.0" || $7 == "0" ' alzheimer_onlyrs_uniq | head

echo -e '\n------------------------------------------------\n' 

awk -v OFS='\t' 'NR == 1 || $6 == "Inf" || $6 == "-Inf"' alzheimer_onlyrs_uniq | head

echo -e '\n------------------------------------------------\n' 

awk -v OFS='\t' 'NR == 1 || $10 == "Inf" || $10 == "-Inf"' alzheimer_onlyrs_uniq | head

SNP	CHR	BP	A1	A2	Z	P	N	FREQ	BETA	SE
rs147711004	19	45337918	A	G	39.3407977311966	0	533233	0.038277824715868	0.0149064	0.000378905
rs41289512	19	45351516	G	C	40.8764662452882	0	537683	0.0473418599363339	0.0171533	0.000419638
rs12972156	19	45387459	G	C	Inf	0	758138	0.160681797192298	0	0.000651065
rs12972970	19	45387596	A	G	Inf	0	758349	0.160831579541649	0	0.000651277
rs34342646	19	45388130	A	G	Inf	0	758277	0.164593868083831	0	0.000658883
rs283815	19	45390333	G	A	Inf	0	696691	0.21932447605506	0	0.000793485
rs6857	19	45392254	T	C	Inf	0	698326	0.18145525060524	0	0.000720893
rs71352238	19	45394336	C	T	Inf	0	762491	0.163301618021702	0	0.000654474
rs184017	19	45394969	G	T	Inf	0	698668	0.219899129622319	0	0.000793398

------------------------------------------------

SNP	CHR	BP	A1	A2	Z	P	N	FREQ	BETA	SE
rs12972156	19	45387459	G	C	Inf	0	758138	0.160681797192298	0	0.000651065
rs12972970	19	45387596	A	G	Inf	0	758349	0.160831579541649	0	0.000651277
rs34342646	19	45388130	A	G	Inf	0	758277	0.164593868

#### Setting P=0 to P=1e-269:

In [47]:
%%bash

awk -v OFS='\t' '$7 == "0"{$7=1e-269} 1' alzheimer_onlyrs_uniq > alzheimer_onlyrs_uniq_p

#### Making file containing the SNPs with infinity z and one that does not contain those SNPs:

In [48]:
%%bash

awk -v OFS='\t' 'NR == 1 || $6 == "Inf" || $6 == "-Inf"' alzheimer_onlyrs_uniq_p > alzheimer_inf
awk -v OFS='\t' '$6 != "Inf" && $6 != "-Inf"' alzheimer_onlyrs_uniq_p > alzheimer_formatted_noInf 


#### Extract true Z-scores from other file:

In [34]:
%%bash

awk 'FNR==NR { FILE1[$6]=$7; next} ($1 in FILE1) {print FILE1[$1], $0}' ~/alzheimersproject/1_raw_data/alzheimer_jansen alzheimer_inf > alzheimer_inf_jansen
awk 'FNR==NR { FILE1[$6]=$13; next} ($2 in FILE1) {print FILE1[$2], $0}' ~/alzheimersproject/1_raw_data/alzheimer_jansen alzheimer_inf_jansen > alzheimer_inf_beta_jansen
awk -v OFS='\t' 'NR!=1 { print $3, $4, $5, $6, $7, $2, $9, $10, $11, $1, $13 }' alzheimer_inf_beta_jansen > alzheimer_true_z_beta


#### Merging the two files in order to insert the true Z-scores:

In [1]:
%%bash

cat alzheimer_formatted_noInf alzheimer_true_z_beta > alzheimer_formatted
wc -l alzheimer_onlyrs_uniq_p alzheimer_formatted

  12451136 alzheimer_onlyrs_uniq_p
  12451135 alzheimer_formatted
  24902271 total


#### Rearranging columns for ldsc:

In [40]:
%%bash
awk -v OFS='\t' 'BEGIN { print "SNP", "CHR", "BP", "A1", "A2", "FREQ", "P", "N", "Z" } NR!=1 { print $1, $2, $3, $4, $5, $9, $7, $8, $6 }' alzheimer_onlyrs_uniq_p_z_beta > ~/alzheimersproject/ldsc_formatted/alzheimer_ldsc_form

#### Rearranging columns for gsmr:

In [38]:
%%bash
awk -v OFS='\t' 'BEGIN { print "SNP", "A1", "A2", "freq", "b", "se", "p", "N" } NR!=1 { print $1, $4, $5, $9, $10, $11, $7, $8 }' alzheimer_onlyrs_uniq_p_z_beta > ~/alzheimersproject/gsmr_formatted/alzheimer_gsmr_form